### Question 2
here I propose adding in a few steps for preprocessing the data - this step is something I see lacking in the current workflow


In [40]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import RFE, RFECV, SelectKBest, chi2
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
import numpy as NP
from scipy import linalg as LA
from numpy.random import standard_normal
import inputparser
# fix random seed for reproducibility
numpy.random.seed(7)

In [41]:
traffic = inputparser.csv_reader('traffic.csv')
traffic = traffic.dropna()
#traffic = traffic[traffic['number_of_mouse_clicks']==True]
print(traffic.shape)
Y = traffic[['is_fraud']]
x = inputparser.x_maker(traffic)

(23085, 25)


In [47]:
### Sunny's data processing

def feature_processing(data):
    x  = data
    
    #normalize data first
    x = StandardScaler().fit_transform(x)
    
    #Removing features with low variance
    #As an example, suppose that we have a dataset with boolean features, 
    #and we want to remove all features that are either one or zero (on or off) in more than 80% of the samples
    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    x = sel.fit_transform(x)
    
    #Univariate feature selection
    #x_new = SelectKBest(chi2, k=2).fit_transform(x_temp, Y)
    
    #PCA 
    def PCA(data, dims_rescaled_data=2):
        """
        returns: data transformed in 2 dims/columns + regenerated original data
        pass in: data as 2D NumPy array
        """
        m, n = data.shape
        # mean center the data
        data -= data.mean(axis=0)
        # calculate the covariance matrix
        R = NP.cov(data, rowvar=False)
        # calculate eigenvectors & eigenvalues of the covariance matrix
        # use 'eigh' rather than 'eig' since R is symmetric, 
        # the performance gain is substantial
        evals, evecs = LA.eigh(R)
        # sort eigenvalue in decreasing order
        idx = NP.argsort(evals)[::-1]
        evecs = evecs[:,idx]
        # sort eigenvectors according to same index
        evals = evals[idx]
        # select the first n eigenvectors (n is desired dimension
        # of rescaled data array, or dims_rescaled_data)
        evecs = evecs[:, :dims_rescaled_data]
        # carry out the transformation on the data using eigenvectors
        # and return the re-scaled data, eigenvalues, and eigenvectors
        return NP.dot(evecs.T, data.T).T, evals, evecs
    
    x = PCA(x)
    return x


    

In [50]:
x_final = feature_processing(x)
x_final

(array([[-4.63995991,  3.43665654],
        [ 2.37843244,  1.19688439],
        [ 1.24304817,  0.71080905],
        ...,
        [ 1.55501748, -0.78854102],
        [ 1.31034464,  0.65479386],
        [ 1.75416927,  0.45529135]]),
 array([4.0662522 , 2.16148248, 1.89602703, 1.70265525, 1.60348649,
        1.34648663, 1.12580168, 1.00101388, 0.98483745, 0.91098189,
        0.81339453, 0.74996045, 0.71254676, 0.67548113, 0.6325222 ,
        0.58180857, 0.47584066, 0.41464117, 0.38586405, 0.34959569,
        0.20758336, 0.13460932, 0.06812347]),
 array([[-0.14568645,  0.28654055],
        [-0.12138704,  0.06022234],
        [-0.40908085, -0.19727229],
        [-0.42635739, -0.18339892],
        [-0.42744896, -0.13006217],
        [-0.15574214,  0.10465623],
        [-0.00147189, -0.39972849],
        [-0.03385299, -0.40127926],
        [-0.01852803,  0.10702875],
        [-0.01702218,  0.09991026],
        [ 0.09002845,  0.05529297],
        [-0.1988666 ,  0.10557894],
        [-0.2733011